# **DATA CLEANING**

## IMPORTS

In [5]:
# Imports
import numpy as np
import pandas as pd
import statistics 

## LOADING

In [6]:
# We load the data and check the first rows
data = pd.read_csv('../data/raw/idealista_viviendas_detalle.csv', )

# We change the names of the columns to make them easier to english and to avoid special characters
data.columns = ["url", "price", "zone","neighborhood", "built_area", "usable_area", "bedrooms", "bathrooms", "floor", "exterior", "elevator", "garage", "storage_room", "balcony", "new", "condition", "year", "heating", "agency", "consumption_value", "consumption_label", "emissions_value", "emissions_label", "description", "error"]
data.head()

,url,price,zone,neighborhood,built_area,usable_area,bedrooms,bathrooms,floor,exterior,...,condition,year,heating,agency,consumption_value,consumption_label,emissions_value,emissions_label,description,error
0,https://www.idealista.com/inmueble/109356873/,440.000,deusto,"La Ribera-Ibarrekolanda, Bilbao",76.0,70.0,2.0,2.0,3.0,exterior,...,Buen estado,2025.0,Individual,ORDUNTE Inmobiliaria,23,A,4,A,"GASTOS DE GESTIÓN de 1,5 por ciento + IVA a ca...",NaN
1,https://www.idealista.com/inmueble/106221410/,442.000,deusto,"La Ribera-Ibarrekolanda, Bilbao",82.0,NaN,2.0,2.0,1.0,exterior,...,NaN,NaN,Central,LOIOLA GESTIÓN INMOBILIARIA,NaN,A,NaN,A,La nueva promoción de Loiola se compone de 42 ...,NaN
2,https://www.idealista.com/inmueble/107750109/,381.000,deusto,"La Ribera-Ibarrekolanda, Bilbao",58.0,NaN,1.0,1.0,4.0,exterior,...,NaN,NaN,Central,LOIOLA GESTIÓN INMOBILIARIA,NaN,A,NaN,A,La nueva promoción de Loiola se compone de 42 ...,NaN
3,https://www.idealista.com/inmueble/106221526/,575.000,deusto,"La Ribera-Ibarrekolanda, Bilbao",104.0,NaN,3.0,2.0,1.0,exterior,...,NaN,NaN,Central,LOIOLA GESTIÓN INMOBILIARIA,NaN,A,NaN,A,La nueva promoción de Loiola se compone de 42 ...,NaN
4,https://www.idealista.com/inmueble/108491309/,306.000,deusto,"La Ribera-Ibarrekolanda, Bilbao",62.0,NaN,1.0,1.0,1.0,exterior,...,NaN,NaN,NaN,Loiola,NaN,A,NaN,A,"60 viviendas de obra nueva de 1, 2 y 3 habitac...",NaN


## CLEANING

In [7]:
# We check the summarize information of the dataset, to see the types of the columns and if there are null values
data.info()

# We remove the column that we will not use
data = data.drop(columns=["error"]) # Url is not useful for our analysis, but we will keep it for now in case we want to check some house manually

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   url                1490 non-null   object 
 1   price              1417 non-null   object 
 2   zone               1490 non-null   object 
 3   neighborhood       1417 non-null   object 
 4   built_area         1417 non-null   float64
 5   usable_area        968 non-null    float64
 6   bedrooms           1407 non-null   float64
 7   bathrooms          1415 non-null   float64
 8   floor              1271 non-null   float64
 9   exterior           1249 non-null   object 
 10  elevator           1392 non-null   object 
 11  garage             371 non-null    object 
 12  storage_room       381 non-null    object 
 13  balcony            780 non-null    object 
 14  new                1407 non-null   object 
 15  condition          1286 non-null   object 
 16  year               759 n

In [8]:
# We check the rows that contain all null values (except the zona column, which is the only one that has no null values)
null_columns = data[data.drop(columns=["zone", "url"]).isnull().all(axis=1)]
# The rows that contain all null values are the houses that have been removed from the website, probably because they 
# have been sold in just a day (the time between getting the url of the houses and getting the detailed information). 

# We get how many houses have been sold in one day per zone, just out of curiosity
print(null_columns['zone'].value_counts())

# As there are more houses in a zone than in others, we get the percentage of houses that have been sold in one day per zone
print((null_columns['zone'].value_counts() / data['zone'].value_counts()).sort_values(ascending=False))
# In some zones, more than 10% of the houses have been sold in one day.

# We drop these rows, as they do not provide any useful information
data = data.dropna(how='all', subset=data.columns.difference(['zone', "url"]))

zone
abando-albia             23
begona-santutxu          10
uribarri                  9
indautxu                  9
rekalde                   6
basurto-zorroza           5
ibaiondo                  5
casco-viejo               3
deusto                    2
otxarkoaga-txurdinaga     1
Name: count, dtype: int64
zone
begona-santutxu          0.113636
abando-albia             0.082143
uribarri                 0.066176
indautxu                 0.053892
casco-viejo              0.046154
basurto-zorroza          0.033113
rekalde                  0.028037
otxarkoaga-txurdinaga    0.027778
ibaiondo                 0.023585
deusto                   0.020619
san-adrian-la-pena            NaN
Name: count, dtype: float64


In [9]:
# We check again the summarize information of the dataset
data.info()

# We check the percentage of null values per column
null_percentage = data.isnull().mean().sort_values(ascending=False)
print(null_percentage)

# We will tranform some columns to make them easier to use, changing their format, data type and filling null values.
# Let's go column by column:

#? Price: we will change the thousands separator and convert it to a numeric value (no null values)
data['price'] = data['price'].str.replace('.', '', regex=False).astype(float)

#? Zone and neighborhood: will transform them to categorical variables (no null values)
data['zone'] = data['zone'].astype('category')
data['neighborhood'] = data['neighborhood'].astype('category')

#? Built area: we will leave it as is, since there data type is correct (no null values)

#? Usable area: we will not do anything for now (null values: 32%)

#? Bedrooms: as there are just 10 null values, we will check them manually
data[data['bedrooms'].isnull()]["url"]
# All of them are studios where people can not live, so we do not take them into account and we will drop them
data = data.dropna(subset=['bedrooms'])
# We convert it to an integer. Finally (no null values)
data['bedrooms'] = data['bedrooms'].astype(int) 

# As we have drop some rows, we check the percentage and number of null values per column again
null_amount = data.isnull().sum().sort_values(ascending=False)
print(null_percentage, null_amount)

#? Bathrooms: will transform them to integer variables (no null values)
data['bathrooms'] = data['bathrooms'].astype(int)

#? Floor:

#? Exterior: binary column (exterior/interior)

#? Elevator, garage, storage_room, balcony: we will transform these columns into binary ones and fill the null values with False, 
#? as it means that the house does not have these features
for col in ['garage', 'storage_room', 'balcony', 'elevator']:
    data[col] = data[col].notnull()
    data[col] = data[col].fillna(False)

#? New: there are just two possible values: "Obra nueva" and "Segunda Mano", so we will transform this column into a binary one
# Also, the new column, as there are just 10 null values, we will check them manually and fill them with the correct value
print(data[data['new'].isnull()]["url"])
data['new'] = data['new'].map({"Obra nueva": True, "Segunda mano": False})


# After checking the urls, we see that all the houses are new, so we fill the null values with True
data['new'] = data['new'].fillna(True)


# Print the value counts of the new column to check that there are no null values
print(data['new'].value_counts())

#? Condition

#? Year: as there are many null values (45%), we will not do anything for now

#? Heating:

#? Agency: in the case of null values, it means that the house is being sold by the owner, so we will fill the null values with "owner"
data['agency'] = data['agency'].fillna("owner")
data['agency'] = data['agency'].astype('category')

#? Consumption value and label:

#? Emisions value and label:

#? Description: in the case of null values, it means that there is no description, so we will fill the null values with an empty string
data['description'] = data['description'].fillna("")


print(data.info())

# We check the percentage of null values per column
null_percentage = data.isnull().mean().sort_values(ascending=False)
print(null_percentage)

<class 'pandas.core.frame.DataFrame'>
Index: 1417 entries, 0 to 1489
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   url                1417 non-null   object 
 1   price              1417 non-null   object 
 2   zone               1417 non-null   object 
 3   neighborhood       1417 non-null   object 
 4   built_area         1417 non-null   float64
 5   usable_area        968 non-null    float64
 6   bedrooms           1407 non-null   float64
 7   bathrooms          1415 non-null   float64
 8   floor              1271 non-null   float64
 9   exterior           1249 non-null   object 
 10  elevator           1392 non-null   object 
 11  garage             371 non-null    object 
 12  storage_room       381 non-null    object 
 13  balcony            780 non-null    object 
 14  new                1407 non-null   object 
 15  condition          1286 non-null   object 
 16  year               759 non-nu

C:\Users\gonzalo.iglesias\AppData\Local\Temp\ipykernel_10376\2201531428.py:53: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['new'] = data['new'].fillna(True)


In [10]:
# Check the amount of houses with exterior orientation
print(data['exterior'].value_counts())

# Print the urls of the houses with null orientation
print(data[data['exterior'].isnull()]["url"])

print(data[data["new"] == True]["exterior"].isnull().sum())


exterior
exterior    1061
interior     183
Name: count, dtype: int64
10      https://www.idealista.com/inmueble/108574469/
14      https://www.idealista.com/inmueble/104105241/
15      https://www.idealista.com/inmueble/108376976/
16      https://www.idealista.com/inmueble/106596850/
24      https://www.idealista.com/inmueble/109237730/
                            ...                      
1439    https://www.idealista.com/inmueble/108638950/
1457    https://www.idealista.com/inmueble/108812275/
1462    https://www.idealista.com/inmueble/108493769/
1485    https://www.idealista.com/inmueble/107389969/
1486    https://www.idealista.com/inmueble/101540464/
Name: url, Length: 163, dtype: object
42


In [11]:
# Check the amount of houses per condition
print(data['condition'].value_counts())

condition
Buen estado    967
A reformar     309
Name: count, dtype: int64


In [12]:
# Check the amount of heating 
print(data['heating'].value_counts())

# Check the amount of houses with missing value in year, and compare each case with the value of the "new" column. We want to see how many of them have "new" as True
heating_data = data[data['heating'].notnull()][['heating', 'new']]

# We check how many of them have "new" as True
print(heating_data['new'].value_counts())

data[data["new"] == True]["heating"].value_counts()


heating
Individual    763
Central       242
Name: count, dtype: int64
new
False    968
True      37
Name: count, dtype: int64


heating
Individual    26
Central       11
Name: count, dtype: int64

In [13]:
# Check the amount of houses with missing value in year, and compare each case with the value of the "new" column. We want to see how many of them have "new" as True
year_data = data[data['year'].isnull()][['year', 'new']]

# We check how many of them have "new" as True
print(year_data['new'].value_counts())

# Print the new column
print(data['new'].value_counts())

new
False    518
True     131
Name: count, dtype: int64
new
False    1276
True      131
Name: count, dtype: int64


"CONDITION" CLEANING

In [14]:
print(data.shape)
print("Numero Condition null: ", data['condition'].isnull().sum())


#Los 131 casos en los que "condition" es null porque la casa es NUEVA:
print("Numero Condition null y New True: ", data[(data['condition'].isnull()) & (data['new'] == True)].shape[0])

#Voy a confirmarlo a mano: 
print(data[data['condition'].isnull()]['url'])
#Efectivamente todas los valores de "condition" que son null, son casas nuevas. 

# Por lo que voy a rellenar esos 131 valores con de la columna "condition" con "Nuevo"
data.loc[data['condition'].isnull(), 'condition'] = 'Nuevo'


print(data.head())

(1407, 24)
Numero Condition null:  131
Numero Condition null y New True:  131
1       https://www.idealista.com/inmueble/106221410/
2       https://www.idealista.com/inmueble/107750109/
3       https://www.idealista.com/inmueble/106221526/
4       https://www.idealista.com/inmueble/108491309/
5       https://www.idealista.com/inmueble/108491397/
                            ...                      
1461    https://www.idealista.com/inmueble/108493897/
1462    https://www.idealista.com/inmueble/108493769/
1463    https://www.idealista.com/inmueble/108493862/
1464    https://www.idealista.com/inmueble/108493896/
1465    https://www.idealista.com/inmueble/108969841/
Name: url, Length: 131, dtype: object
                                             url     price    zone  \
0  https://www.idealista.com/inmueble/109356873/  440000.0  deusto   
1  https://www.idealista.com/inmueble/106221410/  442000.0  deusto   
2  https://www.idealista.com/inmueble/107750109/  381000.0  deusto   
3  https:/

"YEAR" CLEANING

In [15]:
null_percentage = data.isnull().mean().sort_values(ascending=False)
print(null_percentage)

emissions_value      0.490405
consumption_value    0.487562
year                 0.461265
emissions_label      0.345416
consumption_label    0.323383
usable_area          0.316276
heating              0.285714
exterior             0.115849
floor                0.100924
url                  0.000000
new                  0.000000
agency               0.000000
condition            0.000000
storage_room         0.000000
balcony              0.000000
price                0.000000
garage               0.000000
elevator             0.000000
bathrooms            0.000000
bedrooms             0.000000
built_area           0.000000
neighborhood         0.000000
zone                 0.000000
description          0.000000
dtype: float64


In [16]:
pd.set_option('display.max_rows', None) # he puesto esto para que me meustre todas las filas


#hay 649 casas con valor "null" en year.
print(data[data["year"].isnull()].shape[0])

# ver si hay casos en los que Year es null y "New" es true (es decir, es obra nueva)
print(data[(data["year"].isnull()) & (data["new"]==True)].shape[0])

#Hay 131 casos en los que Year es null y es obra nueva (New==True)
#En estos casos vamos a poner el valor del año actual (2025)
data.loc[(data["year"].isnull()) & (data["new"]==True), "year"] = 2025


#En los 518 casos que NO es OBRA NUEVA poner la MEDIANA de la zona (o hacer LINEAR REGRESSION)
# (esto tendré que probar y ver cual da mejores resultados.)


649
131


In [ ]:
#print the url, zone and year of the houses with null year
print(data[data["year"].isnull()][["url", "zone", "year"]])

                                                url                   zone  \
7     https://www.idealista.com/inmueble/109013297/                 deusto   
8     https://www.idealista.com/inmueble/108626918/                 deusto   
9     https://www.idealista.com/inmueble/108471006/                 deusto   
10    https://www.idealista.com/inmueble/108574469/                 deusto   
12    https://www.idealista.com/inmueble/109408699/                 deusto   
17    https://www.idealista.com/inmueble/108298767/                 deusto   
19    https://www.idealista.com/inmueble/109215410/                 deusto   
24    https://www.idealista.com/inmueble/109237730/                 deusto   
25    https://www.idealista.com/inmueble/109191209/                 deusto   
26    https://www.idealista.com/inmueble/102494642/                 deusto   
47    https://www.idealista.com/inmueble/109384768/                 deusto   
55    https://www.idealista.com/inmueble/109153291/             

OPCION1: Mediana (lo hago separado para probarlo, luego lo juntare)

In [18]:

#print(data[data["year"].isnull()]["url"])


dataCp = data.copy()

#store in datacp2 the rows where year is NOT null.
dataCp2 = dataCp[dataCp["year"].notnull()]

#pasar las zonas unicas a un array y calcular las medianas para cada zona
#HACERLO DICCIONARIO
zonas = dataCp2["zone"].unique().tolist()
medianas = []
medias = []
for i in range(len(zonas)):
    zona = zonas[i]
    mediana = statistics.median(dataCp2.loc[dataCp2["zone"]==zona, "year"])
    media = round(statistics.mean(dataCp2.loc[dataCp2["zone"]==zona, "year"]), 0)
    medianas.append(mediana)
    medias.append(media)

print("Zonas:",zonas)
print("Medianas: ", medianas)
print("Media: ", medias)

#Voy a sustituir los valores nulos de year por la mediana ya que es mas restistente a valores atipicos que la media.
#Pero en el caso particular de deusto voy a elegir el valor que me da la media ya que como podemos ver aqui muchas casas son de construccion nueva, 
#haciendo que la mediana de 2025, pero realmente no todas las casas son de ese año por eso creemos que en este caso es mejor usar el mean.

añosDeusto = dataCp2.loc[dataCp2["zone"]=="deusto", "year"]
#print("Deusto: ", añosDeusto)


#sustitumos valores en la data
for i in range(len(zonas)):
    if(zonas[i]=="deusto"):
        data.loc[(data["zone"]==zonas[i]) & (data["year"].isnull()),"year"] = medias[i]
    else:
        data.loc[(data["zone"]==zonas[i]) & (data["year"].isnull()),"year"] = medianas[i]


#comprobacion de que ya no hay ningun valor null en year
print(data[data["year"].isnull()][["url", "zone", "year"]])

#comprobacion del primer url que antes tenia null para ver que ha funcionado
print(data.loc[data["url"]=="https://www.idealista.com/inmueble/109013297/", "year"])




Zonas: ['deusto', 'uribarri', 'otxarkoaga-txurdinaga', 'basurto-zorroza', 'abando-albia', 'indautxu', 'casco-viejo', 'begona-santutxu', 'rekalde', 'ibaiondo', 'san-adrian-la-pena']
Medianas:  [2025.0, 1965.0, 1996.0, 1970.0, 1936.5, 1957.0, 1899.0, 1974.0, 1975.0, 1924.0, 1988.0]
Media:  [2007.0, 1961.0, 1995.0, 1977.0, 1941.0, 1953.0, 1910.0, 1983.0, 1989.0, 1938.0, 1983.0]
Empty DataFrame
Columns: [url, zone, year]
Index: []
7    2007.0
Name: year, dtype: float64


In [19]:
null_percentage = data.isnull().mean().sort_values(ascending=False)
print(null_percentage)
data.head()

emissions_value      0.490405
consumption_value    0.487562
emissions_label      0.345416
consumption_label    0.323383
usable_area          0.316276
heating              0.285714
exterior             0.115849
floor                0.100924
url                  0.000000
new                  0.000000
agency               0.000000
year                 0.000000
condition            0.000000
storage_room         0.000000
balcony              0.000000
price                0.000000
garage               0.000000
elevator             0.000000
bathrooms            0.000000
bedrooms             0.000000
built_area           0.000000
neighborhood         0.000000
zone                 0.000000
description          0.000000
dtype: float64


,url,price,zone,neighborhood,built_area,usable_area,bedrooms,bathrooms,floor,exterior,...,new,condition,year,heating,agency,consumption_value,consumption_label,emissions_value,emissions_label,description
0,https://www.idealista.com/inmueble/109356873/,440000.0,deusto,"La Ribera-Ibarrekolanda, Bilbao",76.0,70.0,2,2,3.0,exterior,...,False,Buen estado,2025.0,Individual,ORDUNTE Inmobiliaria,23,A,4,A,"GASTOS DE GESTIÓN de 1,5 por ciento + IVA a ca..."
1,https://www.idealista.com/inmueble/106221410/,442000.0,deusto,"La Ribera-Ibarrekolanda, Bilbao",82.0,NaN,2,2,1.0,exterior,...,True,Nuevo,2025.0,Central,LOIOLA GESTIÓN INMOBILIARIA,NaN,A,NaN,A,La nueva promoción de Loiola se compone de 42 ...
2,https://www.idealista.com/inmueble/107750109/,381000.0,deusto,"La Ribera-Ibarrekolanda, Bilbao",58.0,NaN,1,1,4.0,exterior,...,True,Nuevo,2025.0,Central,LOIOLA GESTIÓN INMOBILIARIA,NaN,A,NaN,A,La nueva promoción de Loiola se compone de 42 ...
3,https://www.idealista.com/inmueble/106221526/,575000.0,deusto,"La Ribera-Ibarrekolanda, Bilbao",104.0,NaN,3,2,1.0,exterior,...,True,Nuevo,2025.0,Central,LOIOLA GESTIÓN INMOBILIARIA,NaN,A,NaN,A,La nueva promoción de Loiola se compone de 42 ...
4,https://www.idealista.com/inmueble/108491309/,306000.0,deusto,"La Ribera-Ibarrekolanda, Bilbao",62.0,NaN,1,1,1.0,exterior,...,True,Nuevo,2025.0,NaN,Loiola,NaN,A,NaN,A,"60 viviendas de obra nueva de 1, 2 y 3 habitac..."


OPCION2: LINEAR REGRESSION

In [20]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

# ---------------------------
# 1️⃣ Preparar los datos
# ---------------------------
# Supongamos que tu dataframe se llama `data`
# Eliminamos columnas innecesarias
data_clean = data.drop(columns=['url', 'description','floor','exterior', 'heating','usable_area','consumption_label','emissions_label','consumption_value','emissions_value'])

# Convertir columnas binarias a 0/1
binary_cols = ['elevator', 'garage', 'storage_room', 'balcony', 'new']
for col in binary_cols:
    data_clean[col] = data_clean[col].map({True: 1, False: 0, 'True': 1, 'False': 0, 'Nuevo': 1, 'Buen estado': 0})

# Para 'condition', si hay más de 2 categorías, lo convertimos a One-Hot
# Si solo dos categorías (Nuevo/Buen estado) ya lo manejamos arriba

# Columnas categóricas con múltiples categorías
categorical_cols = ['agency',  'zone', 'neighborhood', 'condition']
data_encoded = pd.get_dummies(data_clean, columns=categorical_cols, drop_first=True)


print(data_encoded.head())

# Eliminar filas con NA
data_encoded = data_encoded.dropna()


# Separar variables dependiente e independiente
X = data_encoded.drop(columns=['year'])
y = data_encoded['year']

# ---------------------------
# 2️⃣ Ajustar Linear Regression
# ---------------------------
model = LinearRegression()
model.fit(X, y)

# ---------------------------
# 3️⃣ Predicciones y residuos
# ---------------------------
y_pred = model.predict(X)
residuals = y - y_pred

# ---------------------------
# 4️⃣ Graficar residuos
# ---------------------------
plt.figure(figsize=(8,5))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel("Predicted Year")
plt.ylabel("Residuals")
plt.title("Residuals vs Predicted Year")
plt.show()

# Histograma de residuos
plt.figure(figsize=(6,4))
sns.histplot(residuals, kde=True)
plt.title("Histograma de residuos")
plt.show()

# Q-Q plot para revisar normalidad
sm.qqplot(residuals, line='45')
plt.title("Q-Q plot de residuos")
plt.show()

# ---------------------------
# 5️⃣ Resumen estadístico de residuos
# ---------------------------
print("Media de residuos:", residuals.mean())
print("Desviación estándar de residuos:", residuals.std())




r2 = r2_score(y, y_pred)
mae = mean_absolute_error(y, y_pred)
rmse = np.sqrt(mean_squared_error(y, y_pred))

print("R²:", r2)
print("MAE:", mae)
print("RMSE:", rmse)

Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'seaborn'

### Consumption label cleaning

In [ ]:
# PREPARING THE DATA
data_consumption = data.copy()
print(data_consumption.info())
data_consumption.drop(columns=["consumption_value", "emissions_value", "emissions_label", "url", "description"], inplace=True)

data_consumption = data_consumption.dropna(subset=["consumption_label"])

# FIRST SUBSET: drop rows with any null value
data_consumption_subset1 = data_consumption.dropna(axis=0)

X_consumption_1 = data_consumption_subset1.drop(columns=["consumption_label"])
y_consumption_1 = data_consumption_subset1["consumption_label"]

# SECOND SUBSET:
data_consumption_subset2 = data_consumption.copy()

for col in ["usable_area", "floor", "year"]:
    data_consumption_subset2[col] = data_consumption[col].fillna(data_consumption[col].median())

for col in ["exterior", "heating", "condition"]:
    data_consumption_subset2[col] = data_consumption[col].fillna(data_consumption[col].mode()[0])

X_consumption_2 = data_consumption_subset2.drop(columns=["consumption_label"])
y_consumption_2 = data_consumption_subset2["consumption_label"]

print(X_consumption_2.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1407 entries, 0 to 1489
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   url                1407 non-null   object  
 1   price              1407 non-null   float64 
 2   zone               1407 non-null   category
 3   neighborhood       1407 non-null   category
 4   built_area         1407 non-null   float64 
 5   usable_area        962 non-null    float64 
 6   bedrooms           1407 non-null   int32   
 7   bathrooms          1407 non-null   int32   
 8   floor              1265 non-null   float64 
 9   exterior           1244 non-null   object  
 10  elevator           1407 non-null   bool    
 11  garage             1407 non-null   bool    
 12  storage_room       1407 non-null   bool    
 13  balcony            1407 non-null   bool    
 14  new                1407 non-null   bool    
 15  condition          1407 non-null   object  
 16  year       

In [ ]:
# IMPLEMENTING THE MODEL
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

cat_cols = ["zone", "neighborhood", "exterior", "condition", "heating", "agency"]

# FIRST SUBSET
X_consumption_1encoded = df = pd.get_dummies(X_consumption_1, columns=cat_cols, drop_first=True)
y_consumption_1encoded = LabelEncoder().fit_transform(y_consumption_1)

model = RandomForestClassifier()
accs = cross_val_score(model, X_consumption_1encoded, y_consumption_1encoded, cv=5) #KFold(n_splits=5, shuffle=True, random_state=5

print("Average accuracy FIRST SUBSET:", sum(accs)/len(accs))

# SECOND SUBSET
X_consumption_2encoded = df = pd.get_dummies(X_consumption_2, columns=cat_cols, drop_first=True)
y_consumption_2encoded = LabelEncoder().fit_transform(y_consumption_2)

X_train, X_test, y_train, y_test = train_test_split(X_consumption_2encoded, y_consumption_2encoded, test_size=0.2, stratify=y_consumption_2encoded, random_state=42)

rf = RandomForestClassifier(n_estimators=300, max_depth=None, random_state=42)
rf.fit(X_train, y_train)

print("Average accuracy SECOND SUBSET:", accuracy_score(y_test, rf.predict(X_test)))

accs = cross_val_score(model, X_consumption_2encoded, y_consumption_2encoded, cv=5) #KFold(n_splits=5, shuffle=True, random_state=5

print("Average accuracy SECOND SUBSET (K-FOLDS):", sum(accs)/len(accs))

c:\Users\david.r\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Average accuracy FIRST SUBSET: 0.5834368530020704
Average accuracy SECOND SUBSET: 0.7486910994764397
Average accuracy SECOND SUBSET (K-FOLDS): 0.5776467346376413


## STORE

##  EXTERIOR (Gonzalo)

In [51]:
data_prueba_exterior = data.copy()
data_prueba_exterior.columns


Index(['url', 'price', 'zone', 'neighborhood', 'built_area', 'usable_area',
       'bedrooms', 'bathrooms', 'floor', 'exterior', 'elevator', 'garage',
       'storage_room', 'balcony', 'new', 'condition', 'year', 'heating',
       'agency', 'consumption_value', 'consumption_label', 'emissions_value',
       'emissions_label', 'description'],
      dtype='object')

In [ ]:
# Miramos si en la columna de descrippcion esta la palabra exterior (si tiene la palabra terraza o balcon lo incluimos tambien) 
# (la palabra interior no vale por que hay muchos que lo ponen en la descrippcion refiriendose a cosas de dentro del piso)



#A partir de agosto del 22 todas las casas nuevas deben de ser exteriores -> hacemos que a partir del 2023 incluido si es nulo se pone exterior (esto creo que bien)
print(data_prueba_exterior["exterior"].isnull().sum())
condicion_esnula = data_prueba_exterior["exterior"].isnull()
condicion_año = data_prueba_exterior["year"] >= 2023
print(data_prueba_exterior.loc[condicion_esnula & condicion_año, "url"])
data_prueba_exterior.loc[condicion_esnula & condicion_año, "exterior"] = "exterior"
print(data_prueba_exterior["exterior"].isnull().sum())


#Con los ultimos ya clasification

163
14      https://www.idealista.com/inmueble/104105241/
15      https://www.idealista.com/inmueble/108376976/
16      https://www.idealista.com/inmueble/106596850/
36      https://www.idealista.com/inmueble/107151456/
37      https://www.idealista.com/inmueble/104175500/
38      https://www.idealista.com/inmueble/106596877/
39      https://www.idealista.com/inmueble/107151440/
56      https://www.idealista.com/inmueble/109337060/
59      https://www.idealista.com/inmueble/108748919/
60      https://www.idealista.com/inmueble/108748921/
66      https://www.idealista.com/inmueble/106517920/
950     https://www.idealista.com/inmueble/100666823/
951     https://www.idealista.com/inmueble/108279857/
953     https://www.idealista.com/inmueble/100666785/
976     https://www.idealista.com/inmueble/108279843/
977     https://www.idealista.com/inmueble/108279860/
978     https://www.idealista.com/inmueble/104105273/
1030    https://www.idealista.com/inmueble/106679471/
1031    https://www.idea